There will be many times where you will need to encrypt text for security.

In addition to Python's default ```hashlib``` package, there's a widely-used package called ```cryptography```.

Let's check them out

## Hashlib

Passwords must be stored encrypted. What's normally used for this is a one-way function that encrypts the password into a bit string, which is very hard to reverse engineer. These are called *hash functions*.

```hashlib``` includes secure algorithms such as:
- SHA1
- SHA224
- SHA512
- MD5

Let's look at how you would hash a password using the MD5 algorithm:

In [1]:
import hashlib

password = 'supersecretpassword'

# The first thing we have to do is turn the string into a binary string with the encode() method
bpassword = password.encode()

m = hashlib.md5()

m.update(bpassword)

m.digest()

b'\xbb\xb2\xc5\xe6=.\xf8\x93\x10o\xdd\ryz\xa9z'

***NOTE***: This is not only useful for passwords, you can hash the entire contents of a file if needed.

# Cryptography

```pip install cryptography```

A popular third party module for encryption.

*Symmetric Key Encryption* is a group of encryption algorithms based on shared keys, this is similar to a password that's used to encrypt and decrypt text.

These algorothsm include:
- Advanced Encryption Algorithm (AES)
- Blowfish
- Data Encryption Standard (DES)
- Serpent
- Twofish

As compared to *Asymmetric* encryption, *Symmetric* encryption is faster and more straightforward, but you have to deal with the hassle of sharing the key between creator and reader.

```Fernet``` is an implementation of the popular AES algorithm

In [2]:
# First, you need to generate a key
from cryptography.fernet import Fernet

key = Fernet.generate_key()

In [3]:
key

b'y2ALGlaGdQyOAk5EU9QvIklJTn4eggGZ87G_LTBYkOI='

You need to store this key securely, because you will need it to decrypt your text. 

***NOTE***: If you decide to store it in a file, use the ```binary``` data type.

In [4]:
with open('encryption_key', 'wb') as file:
    file.write(key)

In [5]:
!ls - l

ls: cannot access '-': No such file or directory
ls: cannot access 'l': No such file or directory


In [6]:
with open('encryption_key', 'rb') as key:
    readable_key = key.read()
    # Create a Fernet object for encrypting your data
    encryptor = Fernet(readable_key)

    message_to_encrypt = b"Super mega secret text"

    encrypted_message = encryptor.encrypt(message_to_encrypt)

In [7]:
encrypted_message

b'gAAAAABexwIwnEdgNDrgmji6ODsu5tjT7rMurO__SWYmorPDXWilCtLrRtvW3fGsIOTyzU0k2ZbGN8K3-Rr-Ug9UY1StMqQ8cQF1kfeIMqo3KLb0D44g_X8='

In [8]:
# You can decrypt the data using a Fernet object with the same key
decrypter = Fernet(readable_key)

decrypter.decrypt(encrypted_message)

b'Super mega secret text'

*Asymmetric* encryption uses a pair of keys, a public and a private key.

The *public* key is designed to be widely shared, while the *private* key should only be held by the creator.

The only way you can read files encrypted by the *public* key, is by decrypting them with the *private* key.

This style is widely used, for example the SSH key pairs you use to communicate with GitHub.

A very popular *asymmetric* encryption method is RSA, and ```cryptography``` offers the possibility to create key pairs.

In [9]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa

private_key = rsa.generate_private_key(
    public_exponent=65537, key_size=4096, backend=default_backend())

In [10]:
private_key

In [11]:
public_key = private_key.public_key()

In [12]:
public_key

You can now use the public key to encrypt

In [13]:
message = b"super secret stuff that needs encryption"

In [14]:
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

encrypted_message = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None))

In [15]:
# Use the private key to decrypt the message
decrypted_message = private_key.decrypt(encrypted_message,
                                        padding.OAEP(
                                            mgf=padding.MGF1(
                                                algorithm=hashes.SHA256()),
                                            algorithm=hashes.SHA256(),
                                            label=None))

In [16]:
decrypted_message

b'super secret stuff that needs encryption'